# Cleaning datasets

Author: Andrea Mock
    
With the help of Selenium, I was able to collect data on the faculty employeed at Wellesley college beginning with the 2012-2013 academic year. Although we were able to save the data into csv files, there is still cleaning that needs to be done before the datasets can be used for further analysis. 

This notebook is dedicated to cleaning all the datasets that have been collected in order to continue with the data analysis process.

In [1]:
import pandas as pd
from collections import Counter

# import for plotting later
from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight')

## Part 1: Load data 
All the datasets are loaded into pandas dataframes for cleaning in part 2. 

In [2]:
# load all the datasets
prof_2012 = pd.read_csv('faculty_info2012.csv', index_col=0)
prof_2013 = pd.read_csv('faculty_info2013.csv', index_col=0)
prof_2014 = pd.read_csv('faculty_info2014.csv', index_col=0)
prof_2015 = pd.read_csv('faculty_info2015.csv', index_col=0)
prof_2016 = pd.read_csv('faculty_info2016.csv', index_col=0)
prof_2017 = pd.read_csv('faculty_info2017.csv', index_col=0)
prof_2018 = pd.read_csv('faculty_info2018.csv', index_col=0)
prof_2019 = pd.read_csv('faculty_info2019.csv', index_col=0)

In [3]:
# check out data in one of the datasets 
prof_2017.head()

,name,title,title2,title3,education
0,Rachid Aadnani,Senior Lecturer in Middle Eastern Studies,NaN,NaN,"B.A., Universite Moulay Ismail (Morocco); M.A...."
1,Ama Baafra Abeberese,Assistant Professor of Economics,NaN,NaN,"B.A., Wellesley College; M.A., M.Phil., Ph.D.,..."
2,Kris Adams,Senior Music Performance Faculty in Vocal Jazz,NaN,NaN,"B.M., Berklee College of Music; M.M., New Engl..."
3,Katherine Adams,"Instructor in Physical Education, Recreation a...",NaN,NaN,"B.S., Nazareth College; MS., University of Mas..."
4,Marjorie Agosin,Professor of Spanish,NaN,NaN,"B.A., University of Georgia; M.A., Ph.D., Indi..."


In [4]:
# save all datasets in a list 
prof_dfs = [prof_2012,prof_2013,prof_2014,prof_2015,prof_2016,prof_2017,prof_2018,prof_2019]

## Part 2: Cleaning the data

### Part 2.1 Extracting educational information
Currently the faculty information is listed as a string in our dataframe. However since there are multiple educational institutions, to facilitate the analysis of these institution we createa additional columns to each contain one educational insitution

In [5]:
def profEducationClean(prof):
    """
    given the education of a professor returns a list of educational institution a person attended 
    along with the degree name
    """
    if (prof != None) and type(prof) != float:
        allEdu = prof.split(';') # split at ; indicating the start of new educational institution
        return [item.strip() for item in allEdu]
    return []

In [6]:
def extractEducation(edu_data, num):
    """
    given a dataset and number extracts that particular entry from our education data list
    """
    if len(edu_data) < num:
        return []
    return [item.strip() for item in edu_data[num-1].split(',')]

In [7]:
def cleanEducationData(prof_df):
    """
    given a dataframe including the information on faculty members cleans the educational entries and 
    creates columns that specify each educational institutions, returns modified dataframe
    """ 
    # create a list of educational entries
    prof_df['edu_list']= prof_df['education'].apply(profEducationClean)
    
    # each educational institution gets its own column
    prof_df['edu1']= prof_df['edu_list'].apply(lambda x: extractEducation(x,1))
    prof_df['edu2']= prof_df['edu_list'].apply(lambda x: extractEducation(x,2))
    prof_df['edu3']= prof_df['edu_list'].apply(lambda x: extractEducation(x,3))
    prof_df['edu4']= prof_df['edu_list'].apply(lambda x: extractEducation(x,4))
    
    return prof_df 

Using the functions from above we can extract the educational information for each of our dataframes containing faculty information: 

In [8]:
cleaned_dfs = [cleanEducationData(df) for df in prof_dfs]

In [10]:
# example dataframe after cleaning
cleaned_dfs[0].head()

,name,title,title2,title3,education,edu_list,edu1,edu2,edu3,edu4
0,Rachid Aadnani,Lecturer in Middle Eastern Studies,NaN,NaN,"B.A., Universite Moulay Ismail (Morocco); M.A....","[B.A., Universite Moulay Ismail (Morocco), M.A...","[B.A., Universite Moulay Ismail (Morocco)]","[M.A., Dartmouth College]","[Ph.D., Binghamton University]",[]
1,Brandon Abbs,Visiting Lecturer in Psychology,NaN,NaN,"B.A., University of Maryland; Ph.D., Universit...","[B.A., University of Maryland, Ph.D., Universi...","[B.A., University of Maryland]","[Ph.D., University of Iowa]",[],[]
2,Rana Abdul-Aziz,Lecturer in Middle Eastern Studies,NaN,NaN,"B.A., M.A., Tufts University","[B.A., M.A., Tufts University]","[B.A., M.A., Tufts University]",[],[],[]
3,Kris Adams,Senior Music Performance Faculty in Vocal Jazz,NaN,NaN,"B.M., Berklee College of Music; M.M., New Engl...","[B.M., Berklee College of Music, M.M., New Eng...","[B.M., Berklee College of Music]","[M.M., New England Conservatory of Music]",[],[]
4,Marjorie Agosin,Luella LaMer Slaner Professor in Latin America...,Professor of Spanish,NaN,"B.A., University of Georgia; M.A., Ph.D., Indi...","[B.A., University of Georgia, M.A., Ph.D., Ind...","[B.A., University of Georgia]","[M.A., Ph.D., Indiana University]",[],[]


### Part 2.2 Extract degrees and colleges
Having now seperate columns containing each educational institution a faculty member visited, we can move on to extracting the degrees they optained as well as the college names. However, because some use different labels for the same degree such as B.A. and A.B., which both have the same meaning. 

We use the function eduEntryClean along cleanEducation to seperate the college a degree name from any given educational data entry and in a subsquent step clean the degree and colleges name.
The cleaning process involves given both of these degrees the same name to facilitate working with the data later on. 

In [11]:
# different designations of degrees and colleges
bachelor = ['A.B.','B.A.', 'B.S.','B.Math.','B.M.','B.Mus.', 'B.E.D.','Sc.B.','B.E.','B.F.A.']
masters = ['M.A.','M.Ed.', 'M.Phil.','A.M.','M.S.', 'M.Sc.','M.M.','Ed.M.','M.F.A.','M.P.','M.Mus.','MFA.']
phd = ['Ph.D', 'D.M.A.','Ph.D.','J.D.']
university = ['University','Universite','College', 'School','Institute','Conservatory','Academy', 'Museum']

In [12]:
def eduEntryClean(entry):
    """
    helper function that given an education entry splits it into colleges and degrees and returns a 
    tuple of college names and degree names a person obtained
    """
    colleges = []
    degrees = []
    if len(entry) == 0:
        return ([],[])
    for edu in entry:
        if edu in bachelor:
            degrees.append(edu)
        elif edu in masters:
            degrees.append(edu)
        elif edu in phd:
            degrees.append(edu)
        else:
            for uni in university:
                if uni in edu:
                    colleges.append(edu)
    return (colleges, degrees)

In [13]:
def cleanEducation(person):
    """
    given a person's dataframe entry, goes through all of their education data
    and returns a tuple of all colleges attended and degrees obtained
    """
    eduEntries = [person['edu1'],person['edu2'],person['edu3'],person['edu4']]
    colleges = []
    degrees = []
    for entry in eduEntries:
        clean_entry = eduEntryClean(entry)
        colleges += clean_entry[0]
        degrees += clean_entry[1]
    return (colleges,degrees)

In [14]:
def cleanColleges(collegeList):
    """
    given a list of colleges checks if harvard is in the name and changes the name to harvard university
    """
    cleanedColleges = []
    for college in collegeList:
        if ("Harvard" in college) and college != 'Harvard University Extension School':
            cleanedColleges.append('Harvard University')
        else:
            cleanedColleges.append(college)
    return cleanedColleges

In [15]:
def cleanDegrees(degreeList):
    """
    given a list of degrees renames degrees that are equivalent and returns a list of cleaned degrees
    """
    degreesCleaned = []
    arts_bachelor = ['B.A.', 'A.B.', ]
    music_bachelor = ['B.M.', 'B.Mus.']
    science_bachelor = ['B.S.','Sc.B.']
    arts_master = ['M.A.','A.M.']
    music_master = ['M.M.','M.Mus.']
    science_master = ['M.Sc.','M.S.']
    edu_master = ['M.Ed.','Ed.M.']
    phd = ['Ph.D.','Ph.D']
    mfa = ['MFA.','M.F.A.']
    for degree in degreeList:
        if degree in arts_bachelor:
            degreesCleaned.append('B.A.')
        elif degree in music_bachelor:
            degreesCleaned.append('B.M.')
        elif degree in science_bachelor:
            degreesCleaned.append('B.S.')
        elif degree in arts_master:
            degreesCleaned.append('M.A.')
        elif degree in music_master:
            degreesCleaned.append('M.M.')
        elif degree in science_master:
            degreesCleaned.append('M.S.')
        elif degree in edu_master:
            degreesCleaned.append('M.Ed.')
        elif degree in phd:
            degreesCleaned.append('Ph.D.')
        elif degree in mfa:
            degreesCleaned.append('M.F.A.')
        else:
            degreesCleaned.append(degree)
    return degreesCleaned

In [16]:
def extractDegreeColleges(prof_df):
    # clean education for all rows
    eduData = prof_df.apply(cleanEducation,axis=1)
    # save college and degree data in new columns
    prof_df['colleges'] = eduData.apply(lambda x: x[0])
    prof_df['degrees'] = eduData.apply(lambda x: x[1])
    
    # create new column with cleaned college names
    prof_df['colleges_clean'] = prof_df['colleges'].apply(cleanColleges)
    
    # clean degree names
    prof_df['degrees_cleaned'] = prof_df['degrees'].apply(cleanDegrees)
    return prof_df

In [17]:
# clean the educational data and extract degree and colleges for each faculty member
df_cleaned = [extractDegreeColleges(df) for df in cleaned_dfs]

In [18]:
# example of how a dataframe looks after next cleaning step
df_cleaned[-1].head()

,name,title,title2,title3,education,edu_list,edu1,edu2,edu3,edu4,colleges,degrees,colleges_clean,degrees_cleaned
0,"Aadnani, Rachid",Senior Lecturer in Middle Eastern Studies,NaN,NaN,"B.A., Universite Moulay Ismail (Morocco); M.A....","[B.A., Universite Moulay Ismail (Morocco), M.A...","[B.A., Universite Moulay Ismail (Morocco)]","[M.A., Dartmouth College]","[Ph.D., Binghamton University]",[],"[Universite Moulay Ismail (Morocco), Dartmouth...","[B.A., M.A., Ph.D.]","[Universite Moulay Ismail (Morocco), Dartmouth...","[B.A., M.A., Ph.D.]"
1,"Abeberese, Ama Baafra",Assistant Professor of Economics,NaN,NaN,"B.A., Wellesley College; M.A., M.Phil., Ph.D.,...","[B.A., Wellesley College, M.A., M.Phil., Ph.D....","[B.A., Wellesley College]","[M.A., M.Phil., Ph.D., Columbia University]",[],[],"[Wellesley College, Columbia University]","[B.A., M.A., M.Phil., Ph.D.]","[Wellesley College, Columbia University]","[B.A., M.A., M.Phil., Ph.D.]"
2,"Adams, Kris",Senior Music Performance Faculty in Vocal Jazz,NaN,NaN,"B.M., Berklee College of Music; M.M., New Engl...","[B.M., Berklee College of Music, M.M., New Eng...","[B.M., Berklee College of Music]","[M.M., New England Conservatory of Music]",[],[],"[Berklee College of Music, New England Conserv...","[B.M., M.M.]","[Berklee College of Music, New England Conserv...","[B.M., M.M.]"
3,"Adhikari, Prabal",Visiting Lecturer in Physics,NaN,NaN,"B.A., Grinnell College; Ph.D., University of M...","[B.A., Grinnell College, Ph.D., University of ...","[B.A., Grinnell College]","[Ph.D., University of Maryland]",[],[],"[Grinnell College, University of Maryland]","[B.A., Ph.D.]","[Grinnell College, University of Maryland]","[B.A., Ph.D.]"
4,"Agosin, Marjorie",Professor of Spanish,NaN,NaN,"B.A., University of Georgia; M.A., Ph.D., Indi...","[B.A., University of Georgia, M.A., Ph.D., Ind...","[B.A., University of Georgia]","[M.A., Ph.D., Indiana University]",[],[],"[University of Georgia, Indiana University]","[B.A., M.A., Ph.D.]","[University of Georgia, Indiana University]","[B.A., M.A., Ph.D.]"


## Part 2.3 Cleaning faculty title information
Faculty members may have more than one title, however since we are interested in the department as well as their academic title such as Associate Professor, Assistant Professor etc. we want to extract only the title that is relevant to determining their rank and title. 

In [19]:
def extractTitle(title): 
    """
    given the title of a faculty member extracts their department and official title
    """
    if type(title) != float:
        if ('Performance Faculty' in title) or ('Accompanist/Coach' in title) or ('Music Instructor' in title):
            split_title = title.split(' in ')
            return split_title[0].strip(), 'Music' # clean music instructors
        
        elif ('Instructor in Physical Education' in title):
            split_title = title.split(' in ')
            return split_title[0].strip(), 'Physical Education'
        
        elif (("Lecturer in" in title) or ('Instructor in' in title) 
            or ('Professor in' in title) or ('Fellow in' in title)): 
            # skip this title if any of these keywords appear
            if (("History of Ideas" in title) or ('Critical Thought' in title) or ('the Health Sciences' in title)
             or ('Physical and Natural Sciences' in title) or ("Social Sciences" in title) or ("Humanities" in title)):
                return
            split_title = title.split(' in ')
            return split_title[0].strip(), split_title[-1].strip()
       
        elif 'PERA' in title:  # extract PE coaches
            split_title = title.split(' of ')
            return split_title[0].strip(), 'Physical Education'
        elif "Professor of" in title:
            split_title = title.split(' of ')
            if "Class of" in title: # special case for professors who have 
                return 'Class of ' + split_title[1].strip(), split_title[-1].strip()
            elif ("Social Sciences" in title) or ("Critical Thought" in title) or ("Humanities" in title): 
                return
            else:
                return split_title[0].strip(), split_title[-1].strip()
        elif (('Coach/Collaborative Keyboardist' in title) or ('Conductor' in title)):
            return title,'Music'

In [27]:
def getTitle(person, titles):
    """
    given a person's information extracts their department and title, if not found returns nothing 
    """
    for title in titles:
        if extractTitle(person[title]) != None:
            return extractTitle(person[title])
    # if nothing found print it
    #print(person['name'], person['title'], 'no title found')

In [21]:
def getItem(titles, index):
    if titles != None:
        return titles[index]
    return ''

In [22]:
def cleanProfessorTitle(title):
    """
    given the title of a professor, cleans and returns cleaned title 
    """
    if "Assistant Professor" in title:
        return "Assistant Professor"
    elif "Associate Professor" in title:
        return "Associate Professor"
    elif "PERA" in title:
        return title
    elif "Fellow" in title:
        return "Postdoctoral Fellow"
    elif "Visiting Lecturer" in title:
        return "Visiting Lecturer"
    elif "Professor" in title:
        return "Professor"
    elif "Senior Lecturer" in title:
        return "Senior Lecturer"
    elif (("Coach/Collaborative Keyboardist" in title) or ('Music Instructor' in title) or
          ("Accompanist" in title) or ("Conductor" in title)): # clean music faculty descriptions
        return "Music Performance Faculty"
    else:
        return title

In [23]:
def cleanDepartments(dept):
    """
    given the name of a department returns a cleaned version of the department name
    """
    
    if "Computer Science" in dept: 
        return "Computer Science"
    elif "Physics" in dept:
        return "Physics"
    elif "Chemistry" in dept:
        return "Chemistry"
    elif "Neuroscience" in dept:
        return "Neuroscience"
    elif ("Women's" in dept) or ("Women’s" in dept) or ("Health" in dept):
        return "WGST"
    elif "Chinese" in dept:
        return "Chinese"
    elif "Quantitative Reasoning" in dept:
        return "Mathematics"
    elif "Biological Sciences" in dept:
        return "Biological Sciences"
    elif "Geosciences" in dept:
        return "Geosciences"
    elif "Writing" in dept:
        return "Writing"
    elif "Econ" in dept:
        return "Economics"
    elif "Christian" in dept:
        return "Religion"
    elif "Astronomy" in dept:
        return "Astronomy"
    elif "Peace" in dept:
        return 'Peace & Justice Studies'
    elif "Environment" in dept: 
        return "Environmental Science"
    elif "Art" in dept:
        return "Art"
    elif "Asia" in dept:
        return "Asian Studies"
    elif "Ethics" in dept:
        return 'Philosophy'
    elif "Classic" in dept:
        return "Classical Studies"
    elif "Cognitive and Linguistic Sciences" in dept:
        return "Cognitive & Linguistic Sciences"
    else:
        return dept

In [24]:
def splitDept(dept):
    """
    if a person has more than one department they are part of where and is used to connect both department names 
    return list of departments
    """
    if dept != None and type(dept) != float:
        dept_split = dept.split('and')
        return [item.strip() for item in dept_split]
    return []

In [25]:
def cleanTitlesDept(prof_df):
    # get titles of professors
    prof_titles = prof_df.apply(lambda x: getTitle(x, ['title', 'title2', 'title3']), axis = 1)
    
    # save title and dept in dataframe columns
    prof_df['role'] = prof_titles.apply(lambda x: getItem(x,0))
    prof_df['dept'] = prof_titles.apply(lambda x: getItem(x,1))
    
    # save cleaned faculty titles in new column
    prof_df['role_cleaned']= prof_df['role'].apply(cleanProfessorTitle)
    
    # save cleaned departments in df column
    cleanedDept = prof_df['dept'].apply(cleanDepartments)
    prof_df['cleaned_dept'] = cleanedDept.apply(splitDept)
    
    return prof_df

In [28]:
final_dfs = [cleanTitlesDept(df) for df in df_cleaned]

In [29]:
final_dfs[0].head()

,name,title,title2,title3,education,edu_list,edu1,edu2,edu3,edu4,colleges,degrees,colleges_clean,degrees_cleaned,role,dept,role_cleaned,cleaned_dept
0,Rachid Aadnani,Lecturer in Middle Eastern Studies,NaN,NaN,"B.A., Universite Moulay Ismail (Morocco); M.A....","[B.A., Universite Moulay Ismail (Morocco), M.A...","[B.A., Universite Moulay Ismail (Morocco)]","[M.A., Dartmouth College]","[Ph.D., Binghamton University]",[],"[Universite Moulay Ismail (Morocco), Dartmouth...","[B.A., M.A., Ph.D.]","[Universite Moulay Ismail (Morocco), Dartmouth...","[B.A., M.A., Ph.D.]",Lecturer,Middle Eastern Studies,Lecturer,[Middle Eastern Studies]
1,Brandon Abbs,Visiting Lecturer in Psychology,NaN,NaN,"B.A., University of Maryland; Ph.D., Universit...","[B.A., University of Maryland, Ph.D., Universi...","[B.A., University of Maryland]","[Ph.D., University of Iowa]",[],[],"[University of Maryland, University of Iowa]","[B.A., Ph.D.]","[University of Maryland, University of Iowa]","[B.A., Ph.D.]",Visiting Lecturer,Psychology,Visiting Lecturer,[Psychology]
2,Rana Abdul-Aziz,Lecturer in Middle Eastern Studies,NaN,NaN,"B.A., M.A., Tufts University","[B.A., M.A., Tufts University]","[B.A., M.A., Tufts University]",[],[],[],[Tufts University],"[B.A., M.A.]",[Tufts University],"[B.A., M.A.]",Lecturer,Middle Eastern Studies,Lecturer,[Middle Eastern Studies]
3,Kris Adams,Senior Music Performance Faculty in Vocal Jazz,NaN,NaN,"B.M., Berklee College of Music; M.M., New Engl...","[B.M., Berklee College of Music, M.M., New Eng...","[B.M., Berklee College of Music]","[M.M., New England Conservatory of Music]",[],[],"[Berklee College of Music, New England Conserv...","[B.M., M.M.]","[Berklee College of Music, New England Conserv...","[B.M., M.M.]",Senior Music Performance Faculty,Music,Senior Music Performance Faculty,[Music]
4,Marjorie Agosin,Luella LaMer Slaner Professor in Latin America...,Professor of Spanish,NaN,"B.A., University of Georgia; M.A., Ph.D., Indi...","[B.A., University of Georgia, M.A., Ph.D., Ind...","[B.A., University of Georgia]","[M.A., Ph.D., Indiana University]",[],[],"[University of Georgia, Indiana University]","[B.A., M.A., Ph.D.]","[University of Georgia, Indiana University]","[B.A., M.A., Ph.D.]",Luella LaMer Slaner Professor,Latin American Studies,Professor,[Latin American Studies]


## Part 2.4 Drop columns used for cleaning
Now that we have extracted the information that we need we can drop the unnecessary columns that were only used for cleaning purposes. 

In [31]:
all_df = []
for df in final_dfs:
    df = df.drop(columns=['education', 'edu1', 'edu2', 'edu3', 'edu4', 'colleges', 'degrees', 'role', 'dept'])
    df = df.rename(columns={"edu_list": "education", "colleges_clean": "colleges", "degrees_cleaned": "degrees", 
                           "role_cleaned": "role", "cleaned_dept": "dept"})
    all_df.append(df)

In [33]:
# cleaned dataframe 
all_df[-1].head()

,name,title,title2,title3,education,colleges,degrees,role,dept
0,"Aadnani, Rachid",Senior Lecturer in Middle Eastern Studies,NaN,NaN,"[B.A., Universite Moulay Ismail (Morocco), M.A...","[Universite Moulay Ismail (Morocco), Dartmouth...","[B.A., M.A., Ph.D.]",Senior Lecturer,[Middle Eastern Studies]
1,"Abeberese, Ama Baafra",Assistant Professor of Economics,NaN,NaN,"[B.A., Wellesley College, M.A., M.Phil., Ph.D....","[Wellesley College, Columbia University]","[B.A., M.A., M.Phil., Ph.D.]",Assistant Professor,[Economics]
2,"Adams, Kris",Senior Music Performance Faculty in Vocal Jazz,NaN,NaN,"[B.M., Berklee College of Music, M.M., New Eng...","[Berklee College of Music, New England Conserv...","[B.M., M.M.]",Senior Music Performance Faculty,[Music]
3,"Adhikari, Prabal",Visiting Lecturer in Physics,NaN,NaN,"[B.A., Grinnell College, Ph.D., University of ...","[Grinnell College, University of Maryland]","[B.A., Ph.D.]",Visiting Lecturer,[Physics]
4,"Agosin, Marjorie",Professor of Spanish,NaN,NaN,"[B.A., University of Georgia, M.A., Ph.D., Ind...","[University of Georgia, Indiana University]","[B.A., M.A., Ph.D.]",Professor,[Spanish]


## Part 2.5 Saving cleaned dataframes
Having finished cleaning all the collected data, the final steps involves saving them in json format. The json format allows us to later to work with the cleaned data easier as saving to csv files turns lists into strings, meaning that we would have to go through the process of transforming them back to lists. Also if you decide to use the data to load a database (such as a MongoDB database), the json format makes it a lot easier.

In [34]:
import json

In [36]:
index = 2012
for df in all_df:
    df.to_json('cleaned'+str(index)+'faculty_data.json')
    index +=1

Done! Now we have all our dataframes saved in json files.